# GroupfMRI6a - Use motor rule regions to predict Motor response regions
#### Using ActFlow

#### Takuya Ito
#### 03/01/2019

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group
import nibabel as nib
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [3]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings

#### Load in FC mappings

In [516]:
sourcedir = '/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/'
sourceROIs = np.loadtxt(sourcedir + 'MotorRule_Regions.csv',delimiter=',')
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))
        
overlappingROIs = np.intersect1d(sourceROIs,targetROIs)
unique_sources = []
for roi in sourceROIs:
    if roi in overlappingROIs:
        continue
    else:
        unique_sources.append(roi)

unique_targets = []
for roi in targetROIs:
    #if roi in overlappingROIs:
    #    continue
    #else:
    unique_targets.append(roi)

####
# Step 2 - concatenate indices for unique sources/targets
source_ind = []
for roi in unique_sources:
    roi_ind = np.where(glasser2==roi+1)[0]
    source_ind.extend(roi_ind)
source_ind = np.asarray(source_ind)

target_ind = []
for roi in unique_targets:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

fcmapping = np.zeros((len(source_ind),len(target_ind),len(subjNums)))

scount = 0
for subj in subjNums:
    fcmapping[:,:,scount] = tools_group.loadMotorRuleToMotorOutputFC(subj)
    scount += 1
#
#fcmapping = np.divide(fcmapping,len(subjNums))

In [536]:
fcmapping_tmp = np.mean(fcmapping,axis=2)

## 1.01 Load data for RH and LH responses

In [ ]:
tools_group = reload(tools_group)
nResponses = 2
nRules = 4
nStim = 4
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_input = np.zeros((len(glasser2),nStim,len(subjNums)))
data_task_sr = np.zeros((len(glasser2),nRules,len(subjNums)))
data_task_motor = np.zeros((len(glasser2),nRules,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Left')
    # Load compositional
    data_task_input[:,:,scount] = tools_group.loadInputActivity(subj,inputtype='colorStim')
    data_task_sr[:,:,scount] = tools_group.loadInputActivity(subj,inputtype='srRed')
    data_task_motor[:,:,scount] = tools_group.loadRuleEncoding(subj,rule='Motor')
    scount += 1

In [ ]:
# Construct compositional representations for rmid and rind
data_composition_rh = np.zeros(data_task_rh.shape)
data_composition_lh = np.zeros(data_task_lh.shape)

# data_composition_rh[:,0,:] = (data_task_input[:,0,:] + data_task_sr[:,0,:] + data_task_motor[:,2,:]) #mid
# data_composition_rh[:,1,:] = (data_task_input[:,2,:] + data_task_sr[:,0,:] + data_task_motor[:,2,:]) #rind 
# data_composition_rh[:,0,:] = (data_task_input[:,0,:] + data_task_sr[:,0,:] + data_task_motor[:,2,:]) #mid
# data_composition_rh[:,1,:] = (np.mean(data_task_input[:,1:,:],axis=1) + data_task_sr[:,0,:] + data_task_motor[:,2,:]) #rind 
# data_composition_rh[:,0,:] = (data_task_input[:,0,:] + data_task_sr[:,0,:]) #mid
# data_composition_rh[:,1,:] = (data_task_input[:,0,:] + data_task_sr[:,0,:]) #rind 
# data_composition_rh[:,0,:] = (data_task_motor[:,2,:]) #mid
# data_composition_rh[:,1,:] = (data_task_motor[:,3,:]) #rind 

# # RH composition
# data_composition_rh[:,0,:] = (data_task_sr[:,0,:] + data_task_input[:,0,:] + data_task_motor[:,2,:]) #bothvert #vertvert #rmid
# data_composition_rh[:,1,:] = (data_task_sr[:,0,:] + data_task_input[:,3,:] + data_task_motor[:,2,:]) #bothvert #hinputhinput #rmid 

# # LH composition
# data_composition_lh[:,0,:] = (data_task_sr[:,0,:] + data_task_input[:,0,:] + data_task_motor[:,0,:]) #bothvert #vertvert #lmid
# data_composition_lh[:,1,:] = (data_task_sr[:,0,:] + data_task_input[:,3,:] + data_task_motor[:,0,:]) #bothvert #horihori #lmid 

# RH composition
data_composition_rh[:,0,:] = (data_task_sr[:,0,:] + data_task_motor[:,2,:]) #bothvert #vertvert #rmid
data_composition_rh[:,1,:] = (data_task_sr[:,0,:] + data_task_motor[:,3,:]) #bothvert #hinputhinput #rmid 

# LH composition
data_composition_lh[:,0,:] = (data_task_sr[:,0,:] + data_task_motor[:,0,:]) #bothvert #vertvert #lmid
data_composition_lh[:,1,:] = (data_task_sr[:,0,:] + data_task_motor[:,1,:]) #bothvert #horihori #lmid 

## 1.02 Generate actflow data for RH responses

In [ ]:
actflow_data = np.zeros((len(glasser2),nResponses,len(subjNums)))

bad_rois = []
scount = 0
for subj in subjNums:
#     print 'Subject', subj, '(', scount+1, '/', len(subjNums), ')'
        
#     # Right Finger 1
#     actflow_data[target_ind,0,scount] = np.dot(data_composition_rh[source_ind,0,scount],fcmapping[:,:,scount])
#     # Right Finger 2
#     actflow_data[target_ind,1,scount] = np.dot(data_composition_rh[source_ind,1,scount],fcmapping[:,:,scount])
    # Right Finger 1
    actflow_data[target_ind,0,scount] = np.dot(data_composition_rh[source_ind,0,scount],fcmapping_tmp)
    # Right Finger 2
    actflow_data[target_ind,1,scount] = np.dot(data_composition_rh[source_ind,1,scount],fcmapping_tmp)

    scount += 1
    

## 1.1 Run across subject decoding on right-hand motor responses

In [ ]:
tools_group = reload(tools_group)

In [ ]:
nproc = 20
ncvs = 1

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowata = stats.zscore(actflow_data[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[target_ind,:,:].copy()
flowdata = actflow_data[target_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_rh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [ ]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]



# Run LH predictions

In [ ]:
actflow_data = np.zeros((len(glasser2),nResponses,len(subjNums)))

bad_rois = []
scount = 0
for subj in subjNums:
#     print 'Subject', subj, '(', scount+1, '/', len(subjNums), ')'
        
#     # Right Finger 1
#     actflow_data[target_ind,0,scount] = np.dot(data_composition_lh[source_ind,0,scount],fcmapping[:,:,scount])
#     # Right Finger 2
#     actflow_data[target_ind,1,scount] = np.dot(data_composition_lh[source_ind,1,scount],fcmapping[:,:,scount])
    # Right Finger 1
    actflow_data[target_ind,0,scount] = np.dot(data_composition_lh[source_ind,0,scount],fcmapping_tmp)
    # Right Finger 2_tmp
    actflow_data[target_ind,1,scount] = np.dot(data_composition_lh[source_ind,1,scount],fcmapping_tmp)
    scount += 1
    

## 1.1 Run across subject decoding on right-hand motor responses

In [ ]:
tools_group = reload(tools_group)

In [ ]:
nproc = 20
ncvs = 1

# realdata = stats.zscore(data_task_lh[target_ind,:,:],axis=0).copy()
# flowata = stats.zscore(actflow_data[target_ind,:,:],axis=0).copy()
realdata = data_task_lh[target_ind,:,:].copy()
flowdata = actflow_data[target_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_lh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [ ]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]

